<a href="https://colab.research.google.com/github/Santosh-Gupta/AbstractNet/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers --q
%tensorflow_version 2.x

     |████████████████████████████████| 645kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 42.1MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 3.8MB 36.6MB/s 


In [0]:
import os

import tensorflow as tf
from transformers import TFRobertaModel

logger = tf.get_logger()
logger.info(tf.__version__)

INFO:tensorflow:2.2.0


In [0]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    logger.info('Running with TPUStrategy on TPU {} with {} cores '
                .format(tpu.cluster_spec().as_dict()['worker'],
                        strategy.num_replicas_in_sync))
except Exception:
    raise ValueError
    strategy = tf.distribute.OneDeviceStrategy(device='/cpu:0')
    logger.warning('Failed initializing TPU! Running on CPU')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.57.112.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.112.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:Running with TPUStrategy on TPU ['10.57.112.122:8470'] with 8 cores 


INFO:tensorflow:Running with TPUStrategy on TPU ['10.57.112.122:8470'] with 8 cores 


In [0]:
class TextDataset(tf.data.Dataset):
    feature = {}
    for i in range(512):
        feature['dim_' + str(i)] = tf.io.VarLenFeature(tf.int64)


    def _parse_example(example_proto):
        parsed_example_dict = tf.io.parse_single_example(example_proto, TextDataset.feature)
        parsed_example = [tf.sparse.to_dense(parsed_example_dict['dim_'+str(i)]) for i in range(512)]
        parsed_example = tf.transpose(tf.stack(parsed_example, axis=0), perm=[1, 0])
        return parsed_example


    def _construct_inputs(input_ids):
        input_ids = tf.cast(input_ids, dtype=tf.int32)
        num_papers = tf.shape(input_ids)[0]
        idx_a = tf.random.uniform(minval=0, maxval=num_papers, shape=[], dtype=tf.int32)
        input_a = tf.gather(input_ids, idx_a)
        all_related_papers = tf.gather(input_ids, tf.where(
            tf.logical_not(tf.reduce_all(tf.equal(input_ids, input_a), axis=-1)))[:, 0])

        idx = tf.random.categorical(tf.zeros([1, num_papers-1], dtype=tf.float32), num_samples=4)[0]
        input_b, input_c, input_d, input_e = tf.unstack(tf.gather(all_related_papers, idx),
                                                        num=4,
                                                        axis=0)
        return input_a, input_b, input_c, input_d, input_e


    def _parse_and_create_sample(example_proto):
        input_ids = TextDataset._parse_example(example_proto)
        sample = TextDataset._construct_inputs(input_ids)
        positive_labels = tf.ones([1])
        negative_labels = tf.ones([batch_size])
        return sample, (negative_labels, positive_labels)
    
    def __new__(cls, tfrecords_pattern, epochs, batch_size):
        _options = tf.data.Options()
        _options.experimental_deterministic = False

        tfrecords = tf.data.Dataset.list_files(tfrecords_pattern)
        dataset = tfrecords.interleave(tf.data.TFRecordDataset,
                                       cycle_length=4,
                                       block_length=16,
                                       num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.with_options(_options)
        dataset = dataset.map(cls._parse_and_create_sample,
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.batch(batch_size, drop_remainder=True)
        dataset = dataset.repeat(epochs)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

In [0]:
batch_size = 4 * strategy.num_replicas_in_sync
epochs = 2
lr = 1e-5

dataset = TextDataset('tfrecords/*', epochs=epochs, batch_size=batch_size)

In [0]:
def dot_product(x, y, pairwise=False, name=None):
    if pairwise:
        x = tf.expand_dims(x, axis=1)
    return tf.reduce_sum(tf.multiply(x, y), axis=-1, name=name, keepdims=not pairwise)


def get_model():
    base_model = TFRobertaModel.from_pretrained('allenai/biomed_roberta_base', from_pt=True)
    inputs = [tf.keras.Input(shape=[512], dtype=tf.int32, name='input_{}'.format(i),
                             batch_size=batch_size) for i in ['a', 'b', 'c', 'd', 'e']]
    outputs = [tf.reduce_mean(base_model(x)[0], axis=1) for x in inputs]

    ff1 = tf.keras.layers.Dense(768, activation='tanh', name='ff1')
    ff1_outputs = [ff1(x) for x in outputs]

    mean_related_papers = tf.reduce_mean(tf.concat(ff1_outputs[1:], axis=1), axis=1, keepdims=True)
    ff2 = tf.keras.layers.Dense(768, activation='tanh', name='ff2')
    ff2_output = ff2(mean_related_papers)

    negative_outputs = dot_product(ff1_outputs[0], ff1_outputs[0], pairwise=True, name='negative')
    positive_outputs = dot_product(ff2_output, ff2_output, pairwise=False, name='positive')

    model = tf.keras.Model(inputs=inputs, outputs=[negative_outputs, positive_outputs])
    return model

In [0]:
def positive_loss(_, y_pred):
    y_pred = tf.nn.sigmoid(y_pred)
    y_true = tf.ones([batch_size//strategy.num_replicas_in_sync, 1])
    return tf.losses.binary_crossentropy(y_true, y_pred)


def negative_loss(_, y_pred):
    y_pred = tf.nn.softmax(y_pred)
    y_true = tf.eye(batch_size//strategy.num_replicas_in_sync)
    return tf.losses.categorical_crossentropy(y_true, y_pred)

In [0]:
loss_dict = {
    'tf_op_layer_positive': positive_loss,
    'tf_op_layer_negative': negative_loss
}
with strategy.scope():
    model = get_model()
    model.compile(loss=loss_dict, optimizer=tf.keras.optimizers.Adam(lr))

    callbacks_list = [tf.keras.callbacks.ModelCheckpoint(model_dir + '/weights.{epoch:02d}', save_weights_only=True)]
    model.fit(dataset, epochs=epochs, callbacks=callbacks_list)